In [1]:
import os 
import sys
import torch
import numpy as np
import pandas as pd


%matplotlib inline

In [2]:
import torch
from graph_bridges.models.generative_models.sb import SB
from graph_bridges.configs.graphs.lobster.config_base import get_config_from_file
from graph_bridges.models.backward_rates.backward_rate import GaussianTargetRateImageX0PredEMA

In [3]:
#read the model
config = get_config_from_file("graph", "lobster", "1687884918")
device = torch.device(config.device)
sb = SB()
sb.create_from_config(config, device)

Real Distribution Not Implemented for graph_spins_lobster


In [30]:
config.data.pre

'C:\\Users\\cesar\\Desktop\\Projects\\DiffusiveGenerativeModelling\\Codes\\graph-bridges\\data\\raw\\graph\\lobster\\graph_spins_lobster.cp'

In [4]:
#test dataloaders
databatch = next(sb.data_dataloader.train().__iter__())
x_spins_data = databatch[0]
number_of_paths = x_spins_data.shape[0]
x_spins_noise = sb.target_dataloader.sample(number_of_paths,device)

#test scheduler
time_steps = sb.scheduler.set_timesteps(10,0.01,sinkhorn_iteration=1)
print(time_steps)

# test model
times = time_steps[6] * torch.ones(number_of_paths)
generating_model : GaussianTargetRateImageX0PredEMA
generating_model = sb.past_model
forward_ = generating_model(x_spins_data.squeeze(),times)
forward_stein = generating_model.stein_binary_forward(x_spins_data.squeeze(),times)

# test losses
from graph_bridges.models.losses.estimators import BackwardRatioSteinEstimator
backward_ration_stein_estimator = BackwardRatioSteinEstimator(config,device)

#print(forward_.shape)
#print(forward_stein.shape)
# stuff

sb.pipeline.bridge_config.sampler.num_steps = 20
# test reference process
x_spins_w_noise = sb.reference_process.spins_on_times(x_spins_data.squeeze(), times)

#test pipeline
#for spins_path,times in sb.pipeline.paths_iterator(sb.training_model,sinkhorn_iteration=1):
#    print(spins_path[0].shape)
#    print(times)

tensor([1.0000, 0.8900, 0.7800, 0.6700, 0.5600, 0.4500, 0.3400, 0.2300, 0.1200,
        0.0100, 0.0000], dtype=torch.float64)


In [5]:
forward_stein

tensor([[-5.4209, -6.4083, -0.2763,  ..., -0.2085, -0.1238, -0.3077],
        [-4.8741, -4.2191, -6.2146,  ..., -6.4170, -5.5974, -0.1235],
        [-4.7131, -3.8757, -5.3370,  ..., -0.2219, -0.2299, -0.2548],
        ...,
        [-5.0396, -0.1577, -0.2082,  ..., -0.1615, -0.2136, -0.1706],
        [-5.4492, -0.1496, -0.1706,  ..., -0.2435, -0.2071, -0.2351],
        [-5.9878, -0.1614, -0.0894,  ..., -0.1646, -0.1688, -0.3109]],
       grad_fn=<SelectBackward0>)

In [6]:
sinkhorn_iteration = 0
if sinkhorn_iteration == 0:
    past_model = None
else:
    past_model = sb.past_model
training_model = sb.training_model

times_batch_0 = []
paths_batch_0 = []
for spins_path,times in sb.pipeline.paths_iterator(past_model,sinkhorn_iteration=sinkhorn_iteration):
    paths_batch_0.append(spins_path)
    times_batch_0.append(times)
    break
    
times_batch_1 = []
paths_batch_1 = []
for spins_path,times in sb.pipeline.paths_iterator(training_model,sinkhorn_iteration=sinkhorn_iteration+1):
    paths_batch_1.append(spins_path)
    times_batch_1.append(times)
    break

20it [00:00, 1979.38it/s]
20it [00:00, 125.88it/s]


In [15]:
spins_path = paths_batch_0[0]
number_of_paths = spins_path.shape[0]
number_of_timesteps = spins_path.shape[1]
number_of_spins = spins_path.shape[2]

times_ = times.reshape(number_of_paths*number_of_timesteps)
spins_paths_ = spins_path.reshape(number_of_paths*number_of_timesteps,-1)

In [22]:
config.

BridgeConfig(config_path='C:\\Users\\cesar\\Desktop\\Projects\\DiffusiveGenerativeModelling\\Codes\\graph-bridges\\results\\graph\\lobster\\1687884918\\config.json', model=ModelConfig(name='GaussianTargetRateImageX0PredEMA', ema_decay=0.9999, do_ema=True, ch=28, num_res_blocks=2, num_scales=4, ch_mult=[1, 1, 1, 1], input_channels=1, scale_count_to_put_attn=1, data_min_max=[0, 1], dropout=0.1, skip_rescale=True, time_embed_dim=28, time_scale_factor=1000, fix_logistic=False, initial_dist='gaussian', rate_sigma=6.0, Q_sigma=512.0, time_exponential=3.0, time_base=1.0), data=GraphSpinsDataLoaderConfig(name='GraphSpinsDataLoader', graph_type='lobster', remove=False, training_proportion=0.8, doucet=True, data_path='C:\\Users\\cesar\\Desktop\\Projects\\DiffusiveGenerativeModelling\\Codes\\graph-bridges\\data\\raw\\graph\\lobster', S=2, shape=[1, 1, 45], possible_params_dict={'n': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'p1': [0.7], 'p2': [0.5]}, random_flips=True, data_min_max=[0, 1], type='d

In [29]:
from dataclasses import asdict

asdict(config)

{'config_path': 'C:\\Users\\cesar\\Desktop\\Projects\\DiffusiveGenerativeModelling\\Codes\\graph-bridges\\results\\graph\\lobster\\1687884918\\config.json',
 'model': {'name': 'GaussianTargetRateImageX0PredEMA',
  'ema_decay': 0.9999,
  'do_ema': True,
  'ch': 28,
  'num_res_blocks': 2,
  'num_scales': 4,
  'ch_mult': [1, 1, 1, 1],
  'input_channels': 1,
  'scale_count_to_put_attn': 1,
  'data_min_max': [0, 1],
  'dropout': 0.1,
  'skip_rescale': True,
  'time_embed_dim': 28,
  'time_scale_factor': 1000,
  'fix_logistic': False,
  'initial_dist': 'gaussian',
  'rate_sigma': 6.0,
  'Q_sigma': 512.0,
  'time_exponential': 3.0,
  'time_base': 1.0},
 'data': {'name': 'GraphSpinsDataLoader',
  'graph_type': 'lobster',
  'remove': False,
  'training_proportion': 0.8,
  'doucet': True,
  'data_path': 'C:\\Users\\cesar\\Desktop\\Projects\\DiffusiveGenerativeModelling\\Codes\\graph-bridges\\data\\raw\\graph\\lobster',
  'S': 2,
  'shape': [1, 1, 45],
  'possible_params_dict': {'n': [5, 6, 7, 8,

In [7]:
# test plots
from graph_bridges.utils.plots.sb_plots import sinkhorn_plot

"""
sinkhorn_plot(sinkhorn_iteration=0,
              states_histogram_at_0=0,
              states_histogram_at_1=0,
              backward_histogram=0,
              forward_histogram=0,
              time_=None,
              states_legends=0)
--"""

#test trainer

'\nsinkhorn_plot(sinkhorn_iteration=0,\n              states_histogram_at_0=0,\n              states_histogram_at_1=0,\n              backward_histogram=0,\n              forward_histogram=0,\n              time_=None,\n              states_legends=0)\n--'